In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("주식분석")

LangSmith 추적을 시작합니다.
[프로젝트명]
주식분석


In [7]:
from fastmcp import FastMCP

ModuleNotFoundError: No module named 'fastmcp'

In [3]:
# 필요 라이브러리 임포트
import os
import json
import re
from pydantic import BaseModel, Field
from typing import Annotated, List, Dict
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import tools_condition, ToolNode
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool

In [4]:
######## nodes.py ########
# --- 문서 로드 및 전처리 ---
loader = PyMuPDFLoader("stock_report/[삼성전자]분기보고서(2024.11.14).pdf")
docs = loader.load()

## : 문서 분할(Split Documents) <-----------추후 문서 제목 단위 분할로 변경 필요
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=500)
split_documents = text_splitter.split_documents(docs)

# 중복 제거
unique_documents = []
seen_contents = set()

for doc in split_documents:
    content = doc.page_content.strip()
    if content not in seen_contents:
        seen_contents.add(content)
        unique_documents.append(doc)

print(f"원본 문서 수: {len(split_documents)}")
print(f"중복 제거 후 문서 수: {len(unique_documents)}")

## 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

# 벡터스토어 생성
vectorstore = Chroma.from_documents(documents=split_documents, embedding=embeddings, persist_directory="stock_report/chroma_db")

# 5. 검색기(Retriever) 생성
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

원본 문서 수: 404
중복 제거 후 문서 수: 403


In [5]:
class InputState(TypedDict):
    start_input: str

class QueryLoop:
    def __init__(self, query_list):
        self.queries = query_list
        self.index = 0
        self.results = {}
        self.fallback_queries = []
        self.unresolved_queries = []

    def has_next(self):
        return self.index < len(self.queries)

    def current_query(self):
        return self.queries[self.index]

    def save_result(self, query_key, result_dict):
        self.results[query_key] = result_dict
        self.index += 1

class OverallState(TypedDict):
    start_input: str
    user_input: str
    messages: Annotated[list, add_messages]
    query_list: List[str]
    Stock_Value_dict: dict
    loop: QueryLoop
    current_query_result: dict

In [6]:
# 사용자 입력 노드 정의
def user_input_node(state: OverallState):
    print("================================= calculation stock =================================")
    print("주식 가치를 분석합니다. 궁금하신 주식명을 말씀해주세요.")
    user_input = input("User: ")

    return {
        **state,
        "user_input": user_input,
        "messages": [HumanMessage(content=user_input)],
        "Stock_Value_dict": {},  # 나중에 여기에 데이터 채워짐
    }

In [7]:
def search_query_generation_node(state: OverallState) -> OverallState:
    user_input = state["user_input"]

    base_queries = [
        "연결 기준 당기순이익",
        "발행주식수",
        "현재 주가",
        "자본총계",
        "자유현금흐름",
        "영업이익",
        "가중평균자본비용(WACC)",
        "예상 미래 현금흐름",
        "성장률",
        "주당 배당금",
        "기타 수익 관련 정보"
    ]

    # 사용자의 기업명을 앞에 붙여서 전체 쿼리 생성
    query_list = [f"{user_input} {q}" for q in base_queries]

    return {
        **state,
        "query_list": query_list
    }

In [8]:
def init_query_loop_node(state: OverallState) -> OverallState:
    query_list = state["query_list"]
    loop = QueryLoop(query_list)

    return {
        **state,
        "loop": loop
    }

In [9]:
def has_next_query_node(state: OverallState) -> str:
    if state["loop"].has_next():
        return "continue"
    else:
        return "done"

In [10]:
def process_query_node(state: OverallState) -> OverallState:
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

    loop = state["loop"]
    query = loop.current_query()
    
    print(f"\n🔍 [RAG 1차 검색] {query}")
    
    # --- 1. RAG 1차 검색 ---
    try:
        docs = list({doc.page_content.strip() for doc in retriever.invoke(query)})
        context = "\n\n".join(docs)[:3000]
    except Exception as e:
        print(f"❗ RAG 1차 검색 실패: {e}")
        context = ""

    # --- 2. 쿼리 리파인 (context 기반, LLM 사용) ---
    refined_query = query
    if context.strip():
        refined_query = query_refiner(context=context, original_query=query, llm=llm)
        print(f"🔁 쿼리 리파인: {query} → {refined_query}")

    # --- 3. RAG 2차 검색 ---
    print(f"\n🔍 [RAG 2차 검색] {refined_query}")
    try:
        refined_docs = list({doc.page_content.strip() for doc in retriever.invoke(refined_query)})
        refined_context = "\n\n".join(refined_docs)[:3000]
    except Exception as e:
        print(f"❗ RAG 2차 검색 실패: {e}")
        refined_context = ""

    # --- 4. context 요약 및 key 값 추출 ---
    parsed = summarize_context(refined_context, llm)
    print(f"📄 요약 추출 결과: {parsed}")

    # --- 5. 결과만 저장 (웹 fallback은 다음 노드에서 판단) ---
    return {
        **state,
        "current_query_result": parsed
    }


In [11]:
def query_refiner(context: str, original_query: str, llm: ChatOpenAI) -> str:
    prompt = ChatPromptTemplate.from_messages([
        ("system", """
당신은 기업 보고서 문서를 기반으로 쿼리를 보정하는 전문가입니다.

[입력 쿼리]는 너무 일반적일 수 있으니, [문맥(context)]을 참고하여
실제 보고서에 더 잘 매칭되는 명확한 쿼리로 바꿔주세요.

가능하면 보고서에 자주 나오는 용어 (예: '요약 재무제표', '연결 손익계산서', '현금흐름표', '지배기업 귀속 당기순이익') 등을 반영해주세요.

반드시 보정된 쿼리 한 줄만 출력해주세요.
"""),
        ("human", f"""
[입력 쿼리]
{original_query}

[문맥]
{context[:1500]}

[보정된 쿼리]
""")
    ])

    messages = prompt.format_messages()
    response = llm.invoke(messages)

    return response.content.strip()

In [12]:
def summarize_context(context: str, llm: ChatOpenAI) -> dict:
    prompt = ChatPromptTemplate.from_messages([
        ("system", """
당신은 기업의 재무 데이터를 정리하는 전문 AI입니다.

입력으로 제공된 기업 보고서의 일부(context)를 읽고,  
다음 항목 중 문서에 언급된 값이 있으면 정확히 추출하세요.  
없다면 '없음'이라고 적으세요.

항목:
- Net_Income (당기순이익)
- Operating_income (영업이익)
- Free_cash_flow (자유현금흐름)
- WACC (가중평균자본비용)
- Future_cash_flow (예상 미래 현금 흐름)
- Growth_rate (성장률)
- Dividend_per_share (주당 배당금)
- ROE (자기자본이익률)
- Shares_outstanding (발행 주식 수)
- Stock_price (현재 주가)
- Equity (자기자본)

출력은 반드시 JSON 형식의 파이썬 딕셔너리로 해주세요.
"""),
        ("human", f"""
[문서 내용]
{context[:3000]}
""")
    ])

    messages = prompt.format_messages()
    response = llm.invoke(messages)

    try:
        content = response.content.strip()
        cleaned = re.sub(r"^```(?:json)?\n|\n```$", "", content)
        parsed = json.loads(cleaned)
    except Exception as e:
        print(f"❗ 요약 실패: {e}")
        parsed = {}

    return parsed


In [13]:
def update_loop_state_node(state: OverallState) -> OverallState:
    loop = state["loop"]
    query = loop.current_query()
    parsed = state["current_query_result"]
    stock_data = state["Stock_Value_dict"]

    # 결과 저장 (QueryLoop 내부 results 딕셔너리 업데이트)
    loop.save_result(query_key=query, result_dict=parsed)

    # 기존 stock_data에 병합 (없으면 저장, 이미 있으면 유지)
    for k, v in parsed.items():
        if k not in stock_data or stock_data[k] == "없음":
            stock_data[k] = v

    return {
        **state,
        "loop": loop,
        "Stock_Value_dict": stock_data
    }


In [14]:
def web_search_node(state: OverallState) -> OverallState:
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    tavily = TavilySearchResults(max_results=3)

    loop = state["loop"]
    query = loop.current_query()
    parsed = state["current_query_result"]
    stock_data = state["Stock_Value_dict"]

    print(f"🌐 웹 보완 시도 중: {query}")

    # 1. parsed 중 '없음'인 key가 있는 경우만 보완 시도
    if all(v != "없음" for v in parsed.values()):
        print("✅ 모든 값이 채워져 있음 → 웹 보완 생략")
        return state

    # 2. 웹 검색
    try:
        web_result = tavily.invoke({"query": query})
        web_context = "\n\n".join([
            doc.get("content", "") for doc in (web_result.get("documents", []) if isinstance(web_result, dict) else web_result)
        ])[:3000]

        if web_context.strip():
            web_parsed = summarize_context(web_context, llm)

            # 3. 기존 parsed의 '없음'인 항목만 업데이트
            for k, v in web_parsed.items():
                if parsed.get(k, "없음") == "없음" and v != "없음":
                    parsed[k] = v
                    stock_data[k] = v

            print(f"🧩 웹 검색 보완 결과: {parsed}")
        else:
            print("⚠️ 웹 context 없음")
    except Exception as e:
        print(f"❗ 웹 검색 실패: {e}")

    return {
        **state,
        "current_query_result": parsed,
        "Stock_Value_dict": stock_data
    }


In [15]:
def web_query_refine_node(state: OverallState) -> OverallState:
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    query = state["loop"].current_query()
    parsed = state["current_query_result"]

    # ✅ 모든 값이 채워져 있으면 리파인 생략
    if all(v != "없음" for v in parsed.values()):
        print("✅ 모든 값이 채워져 있음 → 웹 쿼리 리파인 생략")
        return {**state, "refined_web_query": query}

    # ✅ 쿼리 리파인 요청
    prompt = ChatPromptTemplate.from_messages([
        ("system", """
당신은 온라 검색 최적화 전문가입니다.

입력 쿼리를 보고, 검색 엔진에 더 잘 맞도록 **정확하고 구체적인 표현**으로 바꿔주세요.
가능하면 실제 재무 용어나 관련 보고서에서 쓰이는 표현을 포함하세요.

- 숫자 단위나 특정 재무 지표를 명확하게 기술
- "예상" → "전망", "추정", "가이던스"
- "자유현금흐름" → "FCF", "현금흐름표", "순현금흐름"
- 너무 일반적인 단어는 지양 (예: 수익, 실적, 이익)

입력 쿼리:
{query}

보정된 쿼리를 한 줄만 출력하세요.
""")
    ])

    messages = prompt.format_messages(query=query)
    response = llm.invoke(messages)

    refined_query = response.content.strip()

    print(f"🔁 웹 쿼리 재작성: {query} → {refined_query}")

    return {
        **state,
        "refined_web_query": refined_query
    }


In [16]:
def web_search_retry_node(state: OverallState) -> OverallState:
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    tavily = TavilySearchResults(max_results=3)

    refined_query = state.get("refined_web_query", "")
    parsed = state["current_query_result"]
    stock_data = state["Stock_Value_dict"]

    if not refined_query:
        print("❗ refined_web_query 없음 → 웹 재검색 생략")
        return state

    # 보완할 항목이 없는 경우 종료
    if all(v != "없음" for v in parsed.values()):
        print("✅ 모든 항목이 이미 채워짐 → 재검색 불필요")
        return state

    print(f"🌐 보정 쿼리로 웹 재검색: {refined_query}")

    try:
        web_result = tavily.invoke({"query": refined_query})
        web_context = "\n\n".join([
            doc.get("content", "") for doc in (web_result.get("documents", []) if isinstance(web_result, dict) else web_result)
        ])[:3000]

        if web_context.strip():
            web_parsed = summarize_context(web_context, llm)

            # '없음'인 항목만 보완
            for k, v in web_parsed.items():
                if parsed.get(k, "없음") == "없음" and v != "없음":
                    parsed[k] = v
                    stock_data[k] = v

            print(f"✅ 웹 재검색 보완 완료: {parsed}")
        else:
            print("⚠️ 웹 재검색 context 없음")

    except Exception as e:
        print(f"❗ 웹 재검색 실패: {e}")

    return {
        **state,
        "current_query_result": parsed,
        "Stock_Value_dict": stock_data
    }


In [17]:
def calculation_node(state: OverallState) -> OverallState:
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    stock_data = state["Stock_Value_dict"]
    messages = state["messages"]

    prompt = ChatPromptTemplate.from_messages([
        ("system", """
당신은 기업 가치 평가 전문가입니다.

다음은 한 기업의 주요 재무 지표입니다. 이 값을 기반으로 다음을 수행하세요:

1. DCF 방식에 따른 적정 주당 가치 추정  
2. PER 및 PBR 방식의 현재 밸류에이션 분석  
3. 핵심 수치 및 가정 요약  
4. 투자자에게 줄 수 있는 간단한 시사점

출력 형식은 깔끔한 보고서 형식으로, 항목별로 구분해서 설명하세요.
"""),
        ("human", f"""
[입력 재무 지표]
{json.dumps(stock_data, ensure_ascii=False, indent=2)}

이 데이터를 기반으로 평가를 시작하세요.
""")
    ])

    messages = prompt.format_messages()
    response = llm.invoke(messages)

    return {
        **state,
        "messages": messages + [AIMessage(content=response.content)],
        "final_result": {"LLM_summary": response.content}
    }


In [18]:
def end_node(state: OverallState) -> OverallState:
    print("================================= FINAL REPORT =================================")
    
    # LLM이 작성한 분석 요약이 있으면 출력
    final_summary = state.get("final_result", {}).get("LLM_summary", None)
    if final_summary:
        print(final_summary)
    else:
        print("❗최종 분석 결과가 없습니다. 일부 단계에서 문제가 발생했을 수 있습니다.")

    print("✅ 주식 가치 평가가 완료되었습니다.")
    return state


In [22]:
# 1. 전체 상태 정의
workflow = StateGraph(OverallState)

# 2. 순차 노드 추가
workflow.add_node("User Input", user_input_node)
workflow.add_node("Search Query Generation", search_query_generation_node)
workflow.add_node("Init Loop", init_query_loop_node)

workflow.add_node("Process Query", process_query_node)
workflow.add_node("Update State", update_loop_state_node)
workflow.add_node("Web Search", web_search_node)
workflow.add_node("Web Query Refine", web_query_refine_node)
workflow.add_node("Web Search Retry", web_search_retry_node)
workflow.add_node("Calculation", calculation_node)
workflow.add_node("End", end_node)

# 3. 조건 분기 노드 (루프 제어)
workflow.add_conditional_edges(
    "Init Loop",
    has_next_query_node,
    {
        "continue": "Process Query",
        "done": "Calculation"
    }
)

# 4. 노드 연결 정의
workflow.set_entry_point("User Input")
workflow.add_edge("User Input", "Search Query Generation")
workflow.add_edge("Search Query Generation", "Init Loop")

workflow.add_edge("Process Query", "Update State")
workflow.add_edge("Update State", "Web Search")
workflow.add_edge("Web Search", "Web Query Refine")
workflow.add_edge("Web Query Refine", "Web Search Retry")

workflow.add_edge("Calculation", "End")
workflow.add_edge("End", END)

# 5. 그래프 빌드
graph = workflow.compile()

In [26]:
# 초기 상태
initial_state = {
    "start_input": "",
}

# 그래프 실행
final_state = graph.invoke(initial_state)

================================= calculation stock =================================
주식 가치를 분석합니다. 궁금하신 주식명을 말씀해주세요.


User:  삼성전자



🔍 [RAG 1차 검색] 삼성전자 연결 기준 당기순이익
🔁 쿼리 리파인: 삼성전자 연결 기준 당기순이익 → 삼성전자 연결 손익계산서 기준 당기순이익

🔍 [RAG 2차 검색] 삼성전자 연결 손익계산서 기준 당기순이익
📄 요약 추출 결과: {'Net_Income': 261045230, 'Operating_income': 26233258, 'Free_cash_flow': '없음', 'WACC': '없음', 'Future_cash_flow': '없음', 'Growth_rate': '없음', 'Dividend_per_share': '없음', 'ROE': '없음', 'Shares_outstanding': '없음', 'Stock_price': '없음', 'Equity': 386281363}
🌐 웹 보완 시도 중: 삼성전자 발행주식수
🧩 웹 검색 보완 결과: {'Net_Income': 261045230, 'Operating_income': 26233258, 'Free_cash_flow': '없음', 'WACC': '없음', 'Future_cash_flow': '없음', 'Growth_rate': '없음', 'Dividend_per_share': '없음', 'ROE': '없음', 'Shares_outstanding': 6736, 'Stock_price': '없음', 'Equity': 386281363}
🔁 웹 쿼리 재작성: 삼성전자 발행주식수 → 삼성전자 발행주식수 및 주식 구조 분석
❗ refined_web_query 없음 → 웹 재검색 생략
